In [ ]:
import pandas as pd
import numpy as np
# pip install patsy
from patsy import dmatrices, dmatrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=1.5)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
y, X = dmatrices('Survived ~ Age + C(Pclass) + C(Sex)', train, return_type='dataframe')
# flatten y so we can incorporate in sklearn functions
y = np.ravel(y)

In [ ]:
X.head()

In [ ]:
# standardize the features since regularization requires all features to be on same scale
scaler = StandardScaler(copy=True)
# we have created a standardization based on the training data
X_clean = scaler.fit(X).transform(X)

In [ ]:
x_clean2 = pd.DataFrame(X_clean, columns=X.columns)

In [ ]:
x_clean2.head()

In [ ]:
sns.lmplot(x='C(Pclass)[T.3]', y='C(Sex)[T.male]', data=X, ci=None)

In [ ]:
sns.lmplot(x='C(Pclass)[T.3]', y='C(Sex)[T.male]', data=x_clean2, ci=None)

In [ ]:
(337+193)/662

In [ ]:
# build our initial model
lr = LogisticRegression('l1')
lr.fit(X_clean, y)
print (lr.coef_)

In [ ]:
#pd.crosstab(lr.predict(x_clean2),
fin2 = pd.DataFrame([lr.predict(x_clean2),y], index=['Pred','Actual']).T
pd.crosstab(fin2['Pred'],fin2['Actual'])

In [ ]:
# a sparse girl
girl = [1., 0., 0., 0., 21.]
girl = scaler.fit(X).transform(girl)
lr.predict_proba(girl.reshape(1,-1))

In [ ]:
print (girl)

In [ ]:
lr.intercept_

In [ ]:
lr.coef_[0]

In [ ]:
#this is it
logodds = lr.intercept_ +(-0.50102192* -0.55059023)+(-1.29833678*-1.0121585)+(-1.17900718*-1.33473316)+\
(-0.56806052* -0.59550826)
print(logodds)
# convert log-odds to odds
odds = np.exp(logodds)
print('these are the odds '+str(odds))
prob = odds/(1 + odds)
print('these are the probablity of survival '+str(prob))

In [ ]:
def sigmoid(x):
    return 1/(1+np.e**(-x))

In [ ]:
print('these are the probablity of survival...')
sigmoid( np.dot(girl, lr.coef_.T) + lr.intercept_ )

In [ ]:
# compute predicted probability for al=2 using the predict_proba method
lr.predict_proba(girl.reshape(1,-1))[:, 1]

In [ ]:
#copy of X not standardized
XX = X.copy()
#del XX['Intercept']
XX.head()

In [ ]:
# standardize the features since regularization requires all features to be on same scale
scaler = StandardScaler(copy=True)
# we have created a standardization based on the training data
XX = scaler.fit(XX).transform(XX)
pd.DataFrame(XX).head()

In [ ]:
# build our initial model
lr1 = LogisticRegression('l1')
lr1.fit(XX,pd.DataFrame(y) )
print (lr1.coef_)

In [ ]:
sigmoid( np.dot(girl, lr1.coef_.T) + lr1.intercept_ )

In [ ]:
fin = pd.DataFrame([lr1.predict(XX),y], index=['Pred','Actual']).T

In [ ]:
pd.crosstab(fin['Pred'],fin['Actual'])

In [ ]:
337+193+60+72

In [ ]:
(337+193)/662

In [ ]:
# convert odds to probability
prob = odds/(1 + odds)
prob

In [ ]:
b0=-0.9
b1=0.1
rawdata = [21,2,5,31,10,0.34,.23]
test_P=[]

In [ ]:
for data in rawdata:
    P = (np.exp(1)**(b0+b1*data))/(1+np.exp(1)**(b0+b1*data))
    test_P.append(P)

In [ ]:
test_P

In [ ]:
exampl = pd.DataFrame.from_dict({'x':rawdata,'y':tt1})
y4, X4 = dmatrices('y ~ x', exampl, return_type='dataframe')
# flatten y so we can incorporate in sklearn functions
y = np.ravel(y)
lr1 = LogisticRegression(max_iter=1000)
lr1.fit(X4,y4 )
print (lr1.coef_)

In [ ]:
##################     OPTIMIZER   ##########
##minimize log losss

#logistic play
#yz=[.999,0.0004,.1,.66,.2,.0001]
yz1=test_P
tt1=[1,0,0,1,0,0,0]
fin=[]
for tt,yz in zip(tt1,yz1):
    fin.append((tt*np.log(yz) + (1-tt)*np.log(1-yz)))
-1*sum(fin)
#2.9245329377547962
#2.7279212507689738.....b0=-0.9,b1 = 0.1

In [ ]:
np.exp(lr1.predict_log_proba(X4))

In [ ]:
lr1.coef_

In [ ]:
#logistic play
yz=[.999,0.0004,.1,.66,.2,.0001]
tt=[1,0,0,1,0,0]
fin=[]
for tt,yz in zip(tt,yz):
    fin.append((tt*np.log(yz) + (1-tt)*np.log(1-yz)))
-1*sum(fin)